# Generation Models for artificial Social Networks

In [ ]:
# loading igraph
library(igraph)

# loading the random names library
library(randomNames)

### Generating (Social) Networks

In the Social Sciences, we often follow the approach of collecting data on social networks and then analyze
them to learn something about the mechanisms by which groups form or evolve.

Another approach that is more often used in social physics and computer science is to generate artificial networks
according to specific criteria to:

- run simulations about processes within networks
- do experiments by changing parameters in the artificial networks
- investigate the processes by which groups form by formalizing social science principles into mathematical models


there are multiple different network structures that are already built into igraph:

In [ ]:
 # a graph of x vertices where all vertices are connected to all other vertices
g <- make_full_graph(10)
plot(g)

In [ ]:
# a graph in a lattice structure
g <- make_lattice(c(5, 1, 5))
plot(g)

In [ ]:
# a graph in a ring structure
g <- make_ring(10)
plot(g)

In [ ]:
# a graph in a star structure
g <- make_star(10)
plot(g)

In [ ]:
# a graph in a tree structure
g <- make_tree(40,3)
plot(g)

The problem with these simple built-in structures is however, that they do not resample what real social networks look like because they are too structured, are missing random processes,do not exhibit clustering and do not form according to the mechanisms that play a role in the formation of human social networks.

Ideally, we would have an algorithm for generation artificial social networks that very closely mimicks they way in which real networks form

#### Adding the Random Element: Erdos-Renyi graphs

In an Erdos-Renyi graph, we can specifiy a probability and each possible edge is created with that probability.
For example, for an erdos-renyi (or random) graph with probability 0.3, each possible edge is created with a probaility of
30%. This adds a random element to the generation of the graph and two erdos-renyi graphs with the same starting parameters
can look very different based on chance.

In [ ]:
# To create an Erdos-renyi graph, we first create a full graph containing all possible connections between vertices.
FullGraph <- make_full_graph(10)
plot(FullGraph)

In [ ]:
# We then define the percentage of connections that should remain in the graph
p = 0.3

In [ ]:
# We randomly sample a proportion of all edges with probability 1-p and form a vector of edges to be deleted
set.seed(123)
DeletedEdges <- sample(E(FullGraph),(round((1-p)*ecount(FullGraph),0)))

In [ ]:
# We create a graph containing only the leftover edges by deleting all other edges
ErdosRenyi <- delete_edges(FullGraph, DeletedEdges)

In [ ]:
# plotting the graph
set.seed(123)
plot(ErdosRenyi)

In [ ]:
## igraph also has a built-in function for creating Erdos-Renyi graphs
set.seed(123)
ErdosRenyi <- sample_gnp(10,0.3)

set.seed(123)
plot(ErdosRenyi)

#### Adding preferential Attachment: Barabasi-Albert graphs

the idea behind the Barabasi-Albert algorithm is that the probability that a new node will form a connection with each of the existing nodes depends on how many connections the existing nodes already have. 
The more connections the node already has, the more likely it is to form a connection with the new node aswell.

In social science terms, one could say that the new nodes "prefer" nodes that are already "popular".

This is a frequent finding in many observed social network: There are many nodes with relatively few connections
but a few nodes with very many connection

In [ ]:
# The algorithm works by starting out with a basic network structure:
net <- g
set.seed(123)
plot(net)

In [ ]:
# then a new node is added to the graph
set.seed(123)
plot(net + vertex("Newbie"))

To understand the algorithm, we need to express the idea of preferential attachment in mathematical terms:

Lets have a look at the degrees of all the nodes

In [ ]:
#we can get a vector of the degree of all nodes in the network like this:
degree(net)

In [ ]:
# if we sum these degrees we get a number that corresponds to twice the amount of all edges in the network
sum(degree(net))
length(E(net))*2 == sum(degree(net))

In [ ]:
# we can now compute the probability for the Newbie to form a connection to any given node by dividing each nodes degree by the sum
# of the degrees of all nodes

EdgeFormProb <- degree(net)/sum(degree(net))
EdgeFormProb

In [ ]:
# computing the probability in this way is neat, because the sum of all probabilites equals 100%
# which means that the new node will make a connection to a node from the network
sum(EdgeFormProb)

To understand the algorithm better and visualize its progression, lets code our own function
to add new nodes to a network.

The function should take a network as input, take a parameter m as input that determines to
how many existing nodes a new nodes will connect and should output a network with a new node
added based on the preferential attachment probability measure.

In [ ]:
BarabasiAlbert <-function(network, m = 2){
        
        # m needs to be larger than one but smaller or equal to the number of nodes in the initial network
        if(m < 1 | m > length(V(network))){
                
                print("m needs to be larger than one but smaller or equal to the number of nodes in the initial network")
                stop()
        }
        
        
        # First, we need to generate a random name for the new node
        NewNodeName <- randomNames(1,which.names="first")
        
        # Then we check the validity of the name. It must not contain special characters and should not already be used
        # in the network. If this is the case, the loop below will redraw random names until we get a valid one
        while(grepl('[^[:alnum:]]', NewNodeName) == TRUE | is.element(NewNodeName,names(V(network))) == TRUE){
                
                NewNodeName <- randomNames(1,which.names="first")
                
        }
        
        # Now, we are calculating the probability of forming new connections to existing nodes in the network. We get a vector of probabilites
        EdgeFormProb <- degree(network)/sum(degree(network))
        
        # We are using a roulette wheel selection approach by repeating node names as many times as their degree and then sample m times
        # with replacement from the resulting vector
        Roulette <- rep(V(network),degree(network))
        SpinResult <- names(sample(Roulette,m, replace = TRUE))
        
        # Because we could draw the same name twice now, we have to repeat the "spin" until we have m different names
        while(length(SpinResult) != length(unique(SpinResult))){
                
                Roulette <- rep(V(network),degree(network))
                SpinResult <- names(sample(Roulette,m, replace = TRUE))
        }
                
        # forming a vector of edges for the node to be added
        NewSenders <- rep(NewNodeName,length(SpinResult))
                
        # forming a list of new edges
        EdgeList  <- paste(NewSenders, SpinResult)
        EdgeList <- unlist(strsplit(EdgeList," "))
                
        # adding the new node to the network
        network <- network + vertex(NewNodeName)
                
        # adding new edges to the network
        network <- network + edges(EdgeList)
                
        print(paste0("The Node ",NewNodeName," was added to the Network. It formed ", m, " new connections"))
        return(network)
        
}

Lets try out our own function!

First we need an existing network to start with.


NOTE: in the simple function we have created, the vertices must have names and there mustn´t be
ny unconnected nodes.

In [ ]:
# creating initial network to start with
network <- graph(c("Bob","Frank","Frank","Donald","Donald","Bob"), directed = FALSE)
set.seed(123)
plot(network)

Lets add one new node based on the Barabasi-Albert algorithm (we are overwriting the inital graph on each iteration)

In [ ]:
# Adding new node and overwriting old network
network <- BarabasiAlbert(network)
set.seed(123)
plot(network)

Nice! Lets add many nodes according to the Barabasi-Albert Algorithm!

In [ ]:
# number of nodes to be added
x <- 100

In [ ]:
# repeating the function x times (each time, we´re overwriting the inital network) 
for (i in 1:x) {
        
        set.seed(123)
        network <- BarabasiAlbert(network)
        plot(network)
        
}

Lets check the degree distribution of our now large network

In [ ]:
sort(degree(network),decreasing = T)

In [ ]:
# checking the degree distribution
hist(sort(degree(network),decreasing = T),
     breaks = seq(-0.5,35.5,1),
     xlab = "Node Degree",
     main = "Histogram of Node degree")

As we can see from the distribution of node degrees, we have a lot of nodes that have a few connections to others
and very few nodes that have a lot of connections to others.

This is a property that is also observed in many real life social networks and is very different from Erdos-Renyi graphs or the symmetric graph structures from the beginning of this notebook.

With respect to the distribution of node degree, Barabasi-Albert Networks are thus a more accurate representation
of real life social Networks than Erdos-renyi graphs.

NOTE:

Our function is very basic and not optimzed. If you want to generate
Barabasi- Albert Models in the future, igraph has a built-in function for them
which is faster, handles exeptions a lot better and is very well documented.


In [ ]:
# documentation
?sample_pa()

In [ ]:
# generating a graph
g <- sample_pa(100, directed = FALSE)
set.seed(123)
plot(g)

In [ ]:
# checking degree distribution
sort(degree(g),decreasing = T)

hist(sort(degree(g),decreasing = T),
     breaks = seq(-0.5,35.5,1),
     xlab = "Node Degree",
     main = "Histogram of Node degree")

#### Adding similarity attraction / homophily: Homophilic Attachment

In the Barabasi algorithm, we influenced the addition of new nodes to a network
in such a way that new nodes show a "preference" to connect to existing nodes
that already have a lot of connections.

From the social science perspecte, this can be seen as an operationalization of
social proof, meaning that people tend to view other people and things more favorably
when a lot of others already show their interest in it.

Another phenomenon that could be operationalized in the addition of new nodes to a social network
is similarity attraction. People tend to like others who are similar to them more and are more likely to
form relationships with them.

Lets see if you can implement a network generation algorithm that mimics this phenomenon.

In [ ]:
# lets build a very basic network to start with.
Sender <- c("Johanna",
            "Cindy",
            "Olga",
            "Fran",
            "Liza",
            "Cindy")

Nominees <- c("Cindy",
              "Olga",
              "Liza",
              "Johanna",
              "Fran",
              "Fran")

network2 <- data.frame(Sender,Nominees)
net2 <- graph.data.frame(network2, directed=F)

Now we add some features to the nodes so that we have two different groups of vertices.
these features could be any membership in an arbitrary social group.
For the sake of simplicity, lets say we are looking at people with different skin color.

NOTE: having colour names as the group attributes lets us use the vector directly for
coloring the nodes in the plot and we save a step for this example

In [ ]:
# adding a vertex attribute for skin color
V(net2)$group <- c("black","white","white","black","white")

In [ ]:
# lets plot the network
set.seed(123)
plot(net2,vertex.color = V(net2)$group,vertex.label.color = "red",vertex.size = 30)

What we want is an algorithm that contains a parameter for homophily attachment (h),
which influences the likelihood of a new node making a connection to vertices in the network
that have the same group attribute as the new vertex or a different one.

As an example: We want to have an algorithm that lets us specify the likelihood of a new black node
making connections to smiliar (other black) nodes or different (white) nodes. We want the parameter to have such
an influence that we can specify whether a new black node will only form connections with other black nodes (homophily; h = 1), or only form
connections with white nodes (heterophily; h = 0), with a variable probability range in the middle.

Lets build the algorithm on top of the preferential attachment mechanism, so that we can model both processes simultaneously

In [ ]:
# Lets take the Probability metric of the Barabasi-Albert algorithm as a starting point:
EdgeFormProb <- degree(net2)/sum(degree(net2))
EdgeFormProb 

If we only wanted preferential attachment to play a role, a new node would have the highest probability
of forming a connection with Cindy or Fran, because they already have the most existing connections.

However, we want the group attribute to play a role in the attachment probability as well.
We thus have to change the formula for computing the Probability to take into account the group membership
of the nodes and our homophily parameter h:

We change the formula for the edge formation probability, so that we multiply the nominator with the homophily parameter
for the attribute of the same group membership and use 1-h to multiply it with the nominator of different group memberships

Lets have a look at the example network to see what that means with Fran as an example:

In [ ]:
# The probability of a new node forming a connection with Fran independent from the social group is 0.25
EdgeFormProb <- degree(net2)["Fran"]/sum(degree(net2))
EdgeFormProb 

In [ ]:
# lets say we add a new black node and specify h = 1 so that this node will only form connections to other black nodes
set.seed(123)
plot(net2 + vertex("Bob"),vertex.color = c(V(net2)$group,"black"),vertex.label.color = "red",vertex.size = 30)
h = 1

We need to model mathematcally that the probability that the new node will form a new connection to the existing node
is influenced by the group membership depending on the homophily parameter h

In [ ]:
# We first compute the degree of all nodes in the network
Degrees <- degree(net2)
Degrees

In [ ]:
# We now weight the degree of nodes with a similar attribute (black) by multiplying it with h
WeightedSameGroupNodeDegrees <- Degrees[V(net2)$group == "black"]*h
WeightedSameGroupNodeDegrees

In [ ]:
# and weight the degree of nodes with a dissimilar attribute (white) by multiplying it with (h-1)
WeightedOterGroupNodeDegrees <- Degrees[V(net2)$group != "black"]*(1-h)
WeightedOterGroupNodeDegrees

By multiplying in such a way with h = 1, the weighted degree fo Fran and Johanna doesn´t change while the
weighted degree for Cindy, Olga and Liza becomes zero because h-1 = 1-1 = 0.

In [ ]:
# To obtain a vector of probabilities, we now put all Weighted node degrees into a vector that will represent
# our vector of numerators
Num <- c(WeightedSameGroupNodeDegrees,WeightedOterGroupNodeDegrees)
Num

We now need to compute the denominator for our probability vector:
To do this, we multiply all degrees of nodes with the same group attribute by h
and the degree of all nodes with a different attribute by (h-1) and sum the afterwards

In [ ]:
# Computing the Denominator
Den <- sum(Degrees[V(net2)$group == "black"]*h, Degrees[V(net2)$group != "black"]*(1-h))
Den

Now we can simply divide the Numerators through the denominator and have a vector of attachment
probabilities based on preferential attachment and group membership

In [ ]:
# Computing the Probabilites for attachment based on preferential attachment AND homophily (h = 1)
EdgeFormProb <- Num/Den
EdgeFormProb

Note that there is a probability of zero to form connections to white nodes for h=1, only black nodes are considered.
within the black nodes, the new node has a probability of 40% of forming a connection with Johanna but a 60% chance
forming a connection with Fran. This difference is due to Fran having 3 connections already but Johanna just having 2.
We have thus combined prefernetial attachment and similarity attraction in this example. 

In [ ]:
# Building a function that lets us play with different values of h and and display the respective edge form probabilites
# for adding a new node.

TryOutH <- function(h){
        
        Degrees <- degree(net2)
        
        WeightedSameGroupNodeDegrees <- Degrees[V(net2)$group == "black"]*h
        WeightedSameGroupNodeDegrees
        
        WeightedOterGroupNodeDegrees <- Degrees[V(net2)$group != "black"]*(1-h)
        WeightedOterGroupNodeDegrees
        
        Num <- c(WeightedSameGroupNodeDegrees,WeightedOterGroupNodeDegrees)
        
        Den <- sum(Degrees[V(net2)$group == "black"]*h, Degrees[V(net2)$group != "black"]*(1-h))
        
        EdgeFormProb <- Num/Den
        
        EdgeFormProb <- round(EdgeFormProb,3)
        
        set.seed(123)
        plot(net2 + vertex("Bob") + edge("Bob","Fran","Bob","Johanna","Bob","Cindy","Bob","Olga","Bob","Liza"),
             vertex.color = c(V(net2)$group,"black"),
             vertex.label.color = "red",
             vertex.size = 30,
             edge.lty = c(rep(1,6),rep(2,5)),
             edge.width = c(rep(3,6),rep(1,5)),
             edge.label = c(rep(NA,6),EdgeFormProb))
        
}

In [ ]:
# now we can play around with different values of h and see how the edge formation probabilities change for
# different values of h when adding a new node to the network.
TryOutH(0.5)

# Do you notice something special about the value of h = 0.5?

Lets build a function that takes an existing network with some group attributes as input, and
adds new nodes based on preferential attachment + homophily attachment

In [ ]:
AddNewHomophilicNode <-function(network, NewNodeGroup = "random",h = 0.5 ,m = 2){
        
        # Our algorithm depends on a vertex attribute called group, so we need to check whether such an attribute exists
        if(length(V(network)$group) == 0){
                
                print("Vertices need an attribute called group. You can assign it via V(YourNetwork)$group <- c(Your group attributes)")
                stop()
        }
        
        # The algorithm is only meaningfull for parameters of h between 0 and 1, so we only allow h to take this range
        if(h < 0 | h > 1){
                
                print("parameter h must be between 0 and 1")
                stop()
                
        }
        
        # m needs to be larger than one but smaller or equal to the number of nodes in the initial network
        if(m < 1 | m > length(V(network))){
                
                print("m needs to be larger than one but smaller or equal to the number of nodes in the initial network")
                stop()
        }
        
        # First, we need to generate a random name for the new node
        NewNodeName <- randomNames(1,which.names="first")
        
        # Then we check the validity of the name. It must not contain special characters and should not already be used
        # in the network. If this is the case, the loop below will redraw random names until we get a valid one
        
        while(grepl('[^[:alnum:]]', NewNodeName) == TRUE | is.element(NewNodeName,names(V(net2))) == TRUE){
                
                NewNodeName <- randomNames(1,which.names="first")
                
        }
        
        # adding the new node to the network
        network <- network + vertex(NewNodeName)
        
        # by default, our new node is assigned to one of the groups at random.
        # We can however also specify the group of the new node in the function ourselves
        
        if(NewNodeGroup == "random"){
                
                GroupingList <- unique(V(network)$group)
                GroupingList <- GroupingList[is.na(GroupingList) == FALSE]
                
                grouping <- sample(GroupingList,1) 
                V(network)[NewNodeName]$group <- grouping
                
        } else if(NewNodeGroup != "random") {
                
                grouping <- NewNodeGroup
                V(network)[NewNodeName]$group <- grouping
        }
        
        # getting the degree of of all nodes
        Degrees <- degree(network)
        
        # creating a list of numerators for all nodes (multiplying node degree with homophily parameter based on group membership of new node)
        WeightedSameGroupNodeDegrees <- Degrees[V(network)$group == grouping]*h
        WeightedOterGroupNodeDegrees <- Degrees[V(network)$group != grouping]*(1-h)
        
        Num <- c(WeightedSameGroupNodeDegrees,WeightedOterGroupNodeDegrees) 
        
        # denominator
        Den <- sum(Degrees[V(network)$group == grouping]*h, Degrees[V(network)$group != grouping]*(1-h))
        
        # Calculating connection probabilities
        EdgeFormProb <- Num/Den
        
        # Using a roulette wheel algorithm to draw a weighted sample from the existing nodes to form new connections with
        Roulette <- rep(names(EdgeFormProb),round(EdgeFormProb*100000,0))### SOMETHING GOES WRONG HERE
        SpinResult <- sample(Roulette,m, replace = TRUE)
        
        # Because we could draw the same name twice now, we have to repeat the "spin" until we have m different names
        while(length(SpinResult) != length(unique(SpinResult))){
                
                Roulette <- rep(V(network),EdgeFormProb*100)
                SpinResult <- names(sample(Roulette,m, replace = TRUE))
        }
        
        # forming a vector of edges for the node to be added
        NewSenders <- rep(NewNodeName,length(SpinResult))
        
        # forming a list of new edges
        EdgeList  <- paste(NewSenders, SpinResult)
        EdgeList <- unlist(strsplit(EdgeList," "))
        
        # adding new edges to the network
        network <- network + edges(EdgeList)
        
        print(paste0("The Node ",NewNodeName," was added to the Network. It formed ", m, " new connections"))
        return(network)
        
}

Lets try out our function!

In [ ]:
# Adding new Node to the network (NOTE: You are overwriting the old network)
# NOTE: You can repeat this step as many times as you want

net2 <- AddNewHomophilicNode(net2,h = 1, m = 2)
plot(net2,vertex.color = V(net2)$group,vertex.label.color = "red")

Nice! Lets add many nodes to the network!

In [ ]:
# Setting number of nodes to add
x <- 50

In [ ]:
# Adding the nodes and plotting the new network
for (i in 1:x) {
        
        net2 <- AddNewHomophilicNode(net2,h = 1)
        plot(net2,vertex.color = V(net2)$group,vertex.label.color = "red")
        
}


We have succesfully implemented the generation of artificial social networks according to two principles that we can observe
in  the social sciences:

- preferntial attachment
- similarity attraction